<a href="https://colab.research.google.com/github/SiwTurner/Cville_RealEstate/blob/master/Exercises/5-NewsClassification_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils swifter 'tensorflow-text==2.15'

  Using cached keras_nlp-0.11.1-py3-none-any.whl (515 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gensim-4.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.0 MB)
  Using cached np_utils-0.6.0.tar.gz (61 kB)
  Preparing metadata (setup.py) ... done
  Using cached swifter-1.4.0.tar.gz (1.2 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.4 MB/s eta 0:00:00
  Using cached keras_core-0.1.7-py3-none-any.whl (950 kB)
  Using cached namex-0.0.8-py3-none-any.whl (5.8 kB)
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=16b0f46ea71aa3ca46e0b93c095e87d5135756135d23b41b1e003de432ebed60
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=c1f7191c395aa1469118321ebc1fb7c8a87903b864a46ac65dc281b1c7680c13


In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob

TRACE = False
embedding_dim = 300
epochs=100
batch_size = 250
corpus_size=25000
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2024-05-14 13:12:43--  https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/352x7xzivf60zgc/news.csv [following]
--2024-05-14 13:12:43--  https://www.dropbox.com/s/raw/352x7xzivf60zgc/news.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4ccb9c6896eb963ffb4d5ed772.dl.dropboxusercontent.com/cd/0/inline/CS5V2_4k2ifHSrldBMb3BZo2zAR2nugvqgdmWzffe3NbLn4bqf2Y2pgPOwKmMxXLBmo6RTbQorD0ryuxmfqj4Xe1vhSSIqjJIRUphOdpDfFYD71PEgCTpTndmOqp-N18lcsJbSyLYV8x_kjThJkv_mjW/file# [following]
--2024-05-14 13:12:43--  https://uc4ccb9c6896eb963ffb4d5ed772.dl.dropboxusercontent.com/cd/0/inline/CS5V2_4k2ifHSrldBMb3BZo2zAR2nugvqgdmWzffe3NbLn4bqf2Y2pgPOwKmMxXLBmo6RTbQorD0ryuxmfqj4Xe1vhSSIqjJIRUphOdpDfFYD

In [6]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

In [7]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

def preprocess_text(text, should_join=True):
    # Use the tokenizer to tokenize into words, lowercase them, remove punctuation, and finally use gensim.utils.simple_preprocess(text)
    text = ' '.join(word.lower() for word in textblob_tokenizer(text)) #lowercase
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
import swifter
# Use swifter to apply the preprocessin and save that pandas series to a file
news = news_pre.title.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

In [33]:
# prompt: save news pandas series as file

news.to_csv('news_processed.csv', header=False, index=False)


In [34]:

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_processed.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)

from gensim.models import Word2Vec
word2vec_model = gensim.models.Word2Vec(MyCorpus())
# Get a word2vec model using gensim.models and passing the sentences using MyCorpus()

In [35]:
weights = word2vec_model.wv.vectors  # Get the weights of the model (the embedding) and convert to tensor. Hint: Check word2vec_model.wv
vocab_size = len(word2vec_model.wv.index_to_key)  # get vocab size from index_to_key in word2vec_model.wv

In [36]:
weights.shape

(4929, 100)

In [15]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news
news_preprocessed

,label,title
0,0,bbc set for major shake up claims newspaper
1,0,marsh averts cash crunch
2,1,jeter yankees look to take control ap
3,2,flying the sun to safety
4,0,stocks seen flat as nortel and oil weigh
...,...,...
24995,0,wal mart clarifies policy regarding unions ap
24996,2,viewpoint better russian planning would preven...
24997,1,no syracuse princeton
24998,1,usc oklahoma still auburn third in bcs


In [26]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in news_preprocessed.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)   # Since 2 titles may have different number of words, we have to find the max length and fill with 0s if a title is shorter

In [27]:
X = np.zeros((len(news_preprocessed), maximum))   # Here we do what we said above
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# HINT: to iterate through a column of a pandas dataframe you do:

# for index, value in df.iterrows():
#      #do something

# FILL
for row_index, row in news_preprocessed.iterrows():
    for column_index, word in enumerate(textblob_tokenizer(row.title)):
        if word in word2vec_model.wv.index_to_key:
            X[row_index, column_index] = word2vec_model.wv.index_to_key.index(word)
        else:
            X[row_index, column_index] = 0

y = news_preprocessed.label

In [29]:
X[:10].astype(int)

array([[2661,   71,    2,  393, 1808,   15,  178, 3067,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [1411,    0,  723, 3219,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [4304,  282,  316,    0,  128,  639,    5,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [1831,    6,  136,    0, 1225,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [  40,  268,  668,   14,  988,   16,   18, 1127,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [1872, 2155,  218,    0,   32,   62, 2032,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [1002, 2388,  107,   18,   49,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [  52,  107,   86,  726,  106, 1285,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
# Convert y_train and y_test from an array of values between 0-3 to a one hot matrix tensor
y_train = tf.one_hot(y_train, depth=4)
y_test = tf.one_hot(y_test, depth=4)


In [32]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[weights], input_length=maximum, trainable=False))  # Add an Embedding layer with weights being the rweights variable and trainable as False. The embedding dimension should be embedding_dim
model.add(Dense(100, activation='relu'))
model.add(Dense(60, activation='relu')) # Add a couple of Dense Layers with RELU or leaky_relu activations. You may add Batch Norm if you want too
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(None, embedding_dim,)))  # Average out the words of the sentence. The expected out is (N, D) where N is number f samples in batch and D is embedding dimension
model.add(Dense(4, activation='softmax'))  # Add final Dense layer

ValueError: Layer embedding_2 weight shape (4929, 300) is not compatible with provided weight shape (4929, 100).

In [ ]:
# Compile the model. Think what is the best loss to use
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.01)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_text, y_test), callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=2.0)


In [ ]:
# Test with the following two sentences:
# - 'supercomputer will put workers jobless soon'
# - 'patriots goes winning super bowl'

x_val = np.zeros((2, maximum))
for index, row in enumerate(['supercomputer will put workers jobless soon', 'patriots goes winning super bowl']):
    pass
y_val = tf.one_hot([0,1], depth=4)